In [16]:
import numpy as np
import pandas as pd

In [17]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_train.csv')

dropEvent = ['timestamp', 'pageId', 'OS', 'browser', 'device', 'widgetId']
cleanEventData = eventData.drop(columns = dropEvent)

eventData = cleanEventData.sort_values(by = ['displayId']).to_numpy()
clickTrainData = clickTrainData.sort_values(by = ['displayId']).to_numpy()

(evn, evm) = eventData.shape
(trainn, trainm) = clickTrainData.shape

# merging displayId
columnsList = ['displayId', 'adId', 'clicked', 'userId']
(resn, resm) = (trainn, evm + trainm - 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < evn and eventData[curr][0] < clickTrainData[i][0]:
        curr += 1
        
    if curr < evn and clickTrainData[i][0] == eventData[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = clickTrainData[i][j]
            cnt += 1
        for j in range(1, evm):
            result[i][cnt] = eventData[curr][j]
            cnt += 1
    else:
        print('One missed row')

trainDisplayId = pd.DataFrame(data = result, columns = columnsList)
trainDisplayId.head()

,displayId,adId,clicked,userId
0,1210227,7182,0,782749
1,1210227,7125,0,782749
2,1210227,7181,0,782749
3,1210227,535,0,782749
4,1210227,7174,1,782749


In [18]:
adTitle = pd.read_csv('ad_title.csv')

adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
trainForAdId = trainDisplayId.sort_values(by = ['adId']).to_numpy()

numberOfWords = 5
(trainn, trainm) = trainForAdId.shape
(adn, adm) = adTitle.shape

# merging adId
columnsList = ['displayId', 'adId', 'clicked', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
(resn, resm) = (trainn, trainm + numberOfWords)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < adn and adTitle[curr][0] < trainForAdId[i][1]:
        curr += 1
        
    if curr < adn and trainForAdId[i][1] == adTitle[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            if adTitle[curr + j][0] == adTitle[curr][0]:
                result[i][cnt] = adTitle[curr + j][1]
            else:
                result[i][cnt] = 0 #Salam bar tof!
            cnt += 1
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            result[i][cnt] = 0 #Salam bar tof!
            cnt += 1

trainMerged = pd.DataFrame(data = result, columns = columnsList)
trainMerged = trainMerged.sort_values(by = ['displayId'])
trainMerged.head()

,displayId,adId,clicked,userId,wordId0,wordId1,wordId2,wordId3,wordId4
3642243,1210227,7182,0,782749,108,675,734,1006,2313
3639160,1210227,7181,0,782749,853,1006,734,675,108
745066,1210227,535,0,782749,13,16,135,859,0
3625504,1210227,7125,0,782749,2544,1221,912,910,333
3636967,1210227,7174,1,782749,29,109,203,2571,2572


In [19]:
train = trainMerged.copy(deep = True)
train = train.to_numpy()
y_train = []
X_train = []
for i in range(train.shape[0]):
    y_train.append(train[i][2])
    X_train.append([*train[i][:2], *train[i][3:]])
    
columnsList = ['displayId', 'adId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
    
X_train = pd.DataFrame(data = X_train, columns = columnsList)
y_train = pd.DataFrame(data = y_train, columns = ['clicked'])

In [20]:
X_trainWithoutIds = X_train.drop(columns = ['displayId', 'adId'])

# Seyed Version

In [21]:
y_trainArr = y_train.to_numpy()
yTrain = []
for i in range(len(y_trainArr)):
    yTrain.append(y_trainArr[i][0])

X_trainWithoutIds = X_trainWithoutIds.to_numpy()

In [22]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
gnb = CategoricalNB()
gnb.fit(X_trainWithoutIds, yTrain)
#y_pred = ? .predict_proba(X_testWithoutIds)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [6]:
from sklearn.tree import DecisionTreeClassifier
treeModel = DecisionTreeClassifier(random_state=1, max_depth=15)
treeModel.fit(X_trainWithoutIds, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=15, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [23]:
eventData = pd.read_csv('event.csv')
clickTestData = pd.read_csv('click_test.csv')

dropEvent = ['timestamp', 'pageId', 'OS', 'browser', 'device', 'widgetId']
cleanEventData = eventData.drop(columns = dropEvent)

eventData = cleanEventData.sort_values(by = ['displayId']).to_numpy()
clickTestData = clickTestData.sort_values(by = ['displayId']).to_numpy()

(evn, evm) = eventData.shape
(testn, testm) = clickTestData.shape

# merging displayId
columnsList = ['displayId', 'adId', 'userId']
(resn, resm) = (testn, evm + testm - 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(testn):
    while curr < evn and eventData[curr][0] < clickTestData[i][0]:
        curr += 1
        
    if curr < evn and clickTestData[i][0] == eventData[curr][0]:
        cnt = 0
        for j in range(0, testm):
            result[i][cnt] = clickTestData[i][j]
            cnt += 1
        for j in range(1, evm):
            result[i][cnt] = eventData[curr][j]
            cnt += 1
    else:
        print('One missed row')

testDisplayId = pd.DataFrame(data = result, columns = columnsList)
testDisplayId.head()

,displayId,adId,userId
0,151650,7585,10279
1,151650,6257,10279
2,151650,6690,10279
3,151938,7715,17723
4,151938,6690,17723


In [24]:
adTitle = pd.read_csv('ad_title.csv')

adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
testForAdId = testDisplayId.sort_values(by = ['adId']).to_numpy()

numberOfWords = 5
(testn, testm) = testForAdId.shape
(adn, adm) = adTitle.shape

# merging adId
columnsList = ['displayId', 'adId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
(resn, resm) = (testn, testm + numberOfWords)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(testn):
    while curr < adn and adTitle[curr][0] < testForAdId[i][1]:
        curr += 1
        
    if curr < adn and testForAdId[i][1] == adTitle[curr][0]:
        cnt = 0
        for j in range(0, testm):
            result[i][cnt] = testForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            if adTitle[curr + j][0] == adTitle[curr][0]:
                result[i][cnt] = adTitle[curr + j][1]
            else:
                result[i][cnt] = 0 #Salam bar tof!
            cnt += 1
    else:
        cnt = 0
        for j in range(0, testm):
            result[i][cnt] = testForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            result[i][cnt] = 0 #Salam bar tof!
            cnt += 1

testMerged = pd.DataFrame(data = result, columns = columnsList)
testMerged = testMerged.sort_values(by = ['displayId'])
testMerged.head()

,displayId,adId,userId,wordId0,wordId1,wordId2,wordId3,wordId4
468220,151650,6690,10279,2487,2488,574,148,364
1117826,151650,7585,10279,57,151,202,605,0
355417,151650,6257,10279,602,76,2364,0,0
35592,151938,123,17723,21,224,340,341,0
1017470,151938,7379,17723,1849,108,1848,0,0


In [25]:
X_test = testMerged[['userId', 'wordId0', 'wordId1', 'wordId2', 'wordId3', 'wordId4']]
X_test.head()

,userId,wordId0,wordId1,wordId2,wordId3,wordId4
468220,10279,2487,2488,574,148,364
1117826,10279,57,151,202,605,0
355417,10279,602,76,2364,0,0
35592,17723,21,224,340,341,0
1017470,17723,1849,108,1848,0,0


In [28]:
X_testNew = X_test.to_numpy()

In [30]:
X_testNew

array([[ 10279,   2487,   2488,    574,    148,    364],
       [ 10279,     57,    151,    202,    605,      0],
       [ 10279,    602,     76,   2364,      0,      0],
       ...,
       [781785,     13,     16,    135,    859,      0],
       [781785,    108,    675,    734,   1006,   2313],
       [781785,    506,    592,    863,   1172,      0]], dtype=int64)

In [29]:
#predictions = treeModel.predict_proba(X_test)

predictions = gnb.fit(X_trainWithoutIds, yTrain).predict_proba(X_testNew)
#y_pred = ? .predict_proba(X_testWithoutIds)

IndexError: index 2618 is out of bounds for axis 1 with size 2573

In [16]:
for i in range(5):
    print(predictions[i])

[0.73076923 0.26923077]
[0.84205088 0.15794912]
[0.66666667 0.33333333]
[0.8315857 0.1684143]
[0.71960298 0.28039702]


In [15]:
def rankList(probabilities):
    ans = []
    n = len(probabilities)
    for i in range(n):
        cnt = 1
        for j in range(n):
            if i == j:
                continue
            if probabilities[i] < probabilities[j] or (probabilities[i] == probabilities[j] and j < i):
                cnt += 1
        ans.append(cnt)
    return ans

In [17]:
testData = pd.read_csv('click_test.csv').to_numpy()
testPre = testMerged.to_numpy()

(n, m) = testData.shape
(tn, tm) = testPre.shape

rank = []

i = 0

l = 0
while l < tn:
    r = l + 1
    while r < tn and testPre[r][0] == testPre[l][0]:
        r += 1
    List = []
    lki = i
    while i < n and testData[i][0] == testPre[l][0]:
        for j in range(l, r):
            if testPre[j][1] == testData[i][1]:
                List.append(predictions[j][1])
                break
        i += 1
    rankListed = rankList(List)
    for j in range(lki, i):
        rank.append(rankListed[j - lki])
    l = r
Answer = pd.DataFrame(data = testData, columns = ['displayId', 'adId'])
Answer['clicked'] = rank
Answer.head()

,displayId,adId,clicked
0,151650,7585,3
1,151650,6257,1
2,151650,6690,2
3,151938,7454,4
4,151938,7370,6


In [18]:
Answer.to_csv('answerWith5Words.csv',index=False)